In [1]:
from datetime import datetime, timedelta
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import glob

In [6]:
print(pd.__version__)

1.0.1


In [2]:
extension = 'xlsx'
df_final = pd.DataFrame() 
files = [i for i in glob.glob('*.{}'.format(extension))]
for x in files:
        df = pd.read_excel(x, encoding='latin-1')
        df_final = pd.concat([df_final,df]) 

In [3]:
df = df_final

In [4]:
df['date'] = pd.to_datetime(df['date'].astype(str), errors='coerce')

In [5]:
df = df.dropna()

In [6]:
df = df[df.customer_name != 'Cash']

In [7]:
df = df[df.customer_name != 'COUNTER SALE']

In [8]:
df = df[df.customer_name != 'K C K DENTAL(TVM)-DR.']

In [9]:
df.describe()

,billed_quantity,sale_price,cost_price
count,684313.000000,6.843130e+05,6.843130e+05
mean,3.930293,7.640406e+02,6.637633e+02
std,23.811047,5.559463e+03,5.136254e+03
min,-10.000000,2.000000e-02,1.000000e-02
25%,1.000000,7.619000e+01,5.000000e+01
50%,1.000000,1.809500e+02,1.380000e+02
75%,3.000000,3.571400e+02,2.750000e+02
max,10695.000000,1.438095e+06,1.009500e+06


In [10]:
df['profit'] = (df['sale_price'] - df['cost_price'])

In [11]:
df_user = pd.DataFrame(df['customer_name'].unique())
df_user.columns = ['customer_name']

## Creating new columns Recency, Frequency & Profit for a customer

In [12]:
df_max_purchase = df.groupby('customer_name').date.max().reset_index()

In [13]:
df_max_purchase.columns = ['customer_name','MaxPurchaseDate']

In [14]:
df_max_purchase['recency'] = (df_max_purchase['MaxPurchaseDate'].max() - df_max_purchase['MaxPurchaseDate']).dt.days

In [15]:
df_user = pd.merge(df_user, df_max_purchase[['customer_name','recency']], on='customer_name')

In [16]:
df_frequency = df.groupby('customer_name').date.count().reset_index()

In [17]:
df_frequency.columns = ['customer_name','frequency']

In [18]:
df_user = pd.merge(df_user, df_frequency, on='customer_name')

In [19]:
df_profit = df.groupby('customer_name').profit.sum().reset_index()

In [20]:
df_user = pd.merge(df_user, df_profit, on='customer_name')

## Normalization

In [21]:
scaler = MinMaxScaler()

In [22]:
df_user[['recency','frequency','profit']] = scaler.fit_transform(df_user[['recency','frequency','profit']])

In [23]:
#df_user.to_csv('test.csv', index =False)

In [24]:
#df_user = df_user.fillna(0)

In [25]:
#df_user['recency'].isnull().values.any()

## Clustering

In [26]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(df_user[['recency']])
df_user['recency_cluster'] = kmeans.predict(df_user[['recency']])

In [27]:
def order_cluster(cluster_field_name, target_field_name,df_,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df_.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df_,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [28]:
df_user = order_cluster('recency_cluster', 'recency',df_user,False)

In [29]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(df_user[['frequency']])
df_user['frequency_cluster'] = kmeans.predict(df_user[['frequency']])

In [30]:
df_user = order_cluster('frequency_cluster', 'frequency',df_user,True)

In [31]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(df_user[['profit']])
df_user['profit_cluster'] = kmeans.predict(df_user[['profit']])

In [32]:
df_user = order_cluster('profit_cluster', 'profit',df_user,True)

In [33]:
df_user.groupby('recency_cluster')['recency'].describe()

,count,mean,std,min,25%,50%,75%,max
recency_cluster,,,,,,,,
0,947.0,0.798547,0.086912,0.659136,0.741493,0.793537,0.839720,1.000000
1,1074.0,0.516748,0.070002,0.406634,0.459608,0.507578,0.575136,0.655133
2,1450.0,0.294469,0.064991,0.180726,0.238848,0.291679,0.353088,0.406348
3,2244.0,0.064882,0.056337,0.000000,0.009151,0.065485,0.108450,0.180440


In [34]:
df_user.groupby('frequency_cluster')['frequency'].describe()

,count,mean,std,min,25%,50%,75%,max
frequency_cluster,,,,,,,,
0,5158.0,0.007456,0.011027,0.000000,0.000310,0.002017,0.010239,0.048402
1,481.0,0.089204,0.035259,0.048557,0.060348,0.078343,0.111697,0.185541
2,72.0,0.285435,0.080318,0.194384,0.221649,0.263885,0.325977,0.528700
3,4.0,0.839707,0.145002,0.653739,0.776489,0.852544,0.915762,1.000000


## Overall Score

In [35]:
df_user['overall_score'] = df_user['recency_cluster'] + df_user['frequency_cluster'] + 1.33*(df_user['profit_cluster'])

In [36]:
df_user['segment'] = 'Low-Value'
df_user.loc[df_user['overall_score']>2,'segment'] = 'Mid-Value' 
df_user.loc[df_user['overall_score']>4,'segment'] = 'High-Value' 

In [37]:
df_user.head()

,customer_name,recency,frequency,profit,recency_cluster,frequency_cluster,profit_cluster,overall_score,segment
0,"DR,ALIKUTTY,KOTTAKKAL",0.585359,0.049643,0.385074,1,1,1,3.33,Mid-Value
1,"DR.REJITH MANGALASSERY, MANANTHAVADY",0.585359,0.123487,0.376859,1,1,1,3.33,Mid-Value
2,"DR.MOYNUDHEEN, PULIKKAL",0.585073,0.068725,0.384385,1,1,1,3.33,Mid-Value
3,DR.WASEEM.MAHE.,0.452960,0.071052,0.380403,1,1,1,3.33,Mid-Value
4,"DR. MANJU, SULTHAN BATHERY",0.599371,0.050884,0.383345,1,1,1,3.33,Mid-Value


In [38]:
df_user.to_csv('nonout_cust_seg.csv', index=False)